In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('trainning_set_drop.csv')

In [3]:
# df = df.dropna(axis = 1)

In [4]:
df.shape

(1460, 71)

In [5]:
df['SalePrice'].describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

In [6]:
df = pd.get_dummies(df, drop_first = True)
df.shape

(1460, 178)

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


# Split the DataFrame into features (X) and target (y)
X = df.drop(['SalePrice'], axis=1)  # Drop the target column to create features
y = df['SalePrice']                   # Target variable

# Split the data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

MMS = MinMaxScaler()


X_train_std = MMS.fit_transform(X_train)
X_test_std = MMS.transform(X_test)

In [8]:
from sklearn.pipeline import Pipeline

In [9]:
# Import necessary libraries
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler


In [10]:
def matrices_regression(y_test, y_pred):
    print('MSE', mean_squared_error(y_test, y_pred))
    
    print('MAE', mean_absolute_error(y_test, y_pred))
    
    
    # print('RMSE', np.sqrt(mean_squared_error(y_test, y_pred)))
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    
    print('R2 Score', r2_score(y_test, y_pred))
    
    n= len(X_train)
    p = len(X_train.columns)
    adj_R2 = 1- ((1-r2_score(y_test, y_pred)) * (n-1)/(n-p-1))
    print('Adjusted R2 Score', adj_R2 )



In [11]:
rfr = RandomForestRegressor()

# Define hyperparameters to search
param_grid = {
    'n_estimators': np.arange(50,350, 25),  # Number of trees
    # 'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 8, 10],  # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 4, 6, 8 , 10],    # Minimum number of samples required at each leaf node
    'max_features': ['auto', 'sqrt', 'log2'],  # Number of features to consider when looking for the best split
    'oob_score': [True]
}

# Perform GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(estimator=rfr, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the model
grid_search.fit(X_train_std, y_train)

Fitting 5 folds for each of 864 candidates, totalling 4320 fits
[CV] END max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50, oob_score=True; total time=   0.0s
[CV] END max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=275, oob_score=True; total time=   0.0s
[CV] END max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=300, oob_score=True; total time=   0.0s
[CV] END max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=300, oob_score=True; total time=   0.0s
[CV] END max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=75, oob_score=True; total time=   0.0s
[CV] END max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=75, oob_score=True; total time=   0.0s
[CV] END max_features=auto, min_samples_leaf=1, min_samples_split=8, n_estimators=100, oob_score=True; total time=   0.0s
[CV] END max_features=auto, min_samples_leaf=1, min_samples_split=8, n_estimators=100

/Users/ghanshyamsingh/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: 
1440 fits failed out of a total of 4320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
844 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/ghanshyamsingh/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 889, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/ghanshyamsingh/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 1507, in wrapper
    estimator._validate_params()
  File "/Users/ghanshyamsingh/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 666, in _validate_params
    valida

GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4, 6, 8, 10],
                         'min_samples_split': [2, 5, 8, 10],
                         'n_estimators': array([ 50,  75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325]),
                         'oob_score': [True]},
             verbose=2)

In [13]:
# Get the best parameters
best_params = grid_search.best_params_
print("Best Parameters found by GridSearchCV:", best_params)
# Make predictions using the best model
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test_std)

# Calculate RMSE

Best Parameters found by GridSearchCV: {'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 250, 'oob_score': True}


In [14]:
matrices_regression(y_test, y_pred)


MSE 1084526605.2417808
MAE 17648.714739726027
Root Mean Squared Error (RMSE): 32932.15154285824
R2 Score 0.8503609373332742
Adjusted R2 Score 0.8124152151384169


In [15]:
import xgboost

In [16]:
xgb_r = xgboost.XGBRegressor()
xgb_r.fit(X_train_std, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [18]:
# Get the best parameters
best_params = grid_search.best_params_
print("Best Parameters found by GridSearchCV:", best_params)
# Make predictions using the best model
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test_std)
y_pred = xgb_r.predict(X_test_std)

In [19]:
matrices_regression(y_test, y_pred)

MSE 1087043200.0
MAE 18628.193359375
Root Mean Squared Error (RMSE): 32970.33818449547
R2 Score 0.8500137329101562
Adjusted R2 Score 0.8119799660406687


In [21]:
import pandas as pd
import optuna
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler



# Define a function to evaluate and print model performance
def evaluate_model(model_name, y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)  # RMSE
    r2 = r2_score(y_true, y_pred)
    explained_variance = explained_variance_score(y_true, y_pred)

    print(f"{model_name} Performance:")
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
    print(f"R-squared (R²): {r2:.2f}")
    print(f"Explained Variance Score: {explained_variance:.2f}")
    print("\n")

# Hyperparameter tuning function using Optuna
def tune_random_forest(trial):
    model = RandomForestRegressor(
        n_estimators=trial.suggest_int('n_estimators', 50, 200),
        max_depth=trial.suggest_int('max_depth', 2, 20),
        min_samples_split=trial.suggest_int('min_samples_split', 2, 20),
        min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 20),
        random_state=42
    )
    return evaluate_model_performance(model)

def tune_xgboost(trial):
    model = XGBRegressor(
        n_estimators=trial.suggest_int('n_estimators', 50, 200),
        max_depth=trial.suggest_int('max_depth', 3, 15),
        learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3),
        random_state=42
    )
    return evaluate_model_performance(model)

def tune_decision_tree(trial):
    model = DecisionTreeRegressor(
        max_depth=trial.suggest_int('max_depth', 1, 20),
        min_samples_split=trial.suggest_int('min_samples_split', 2, 20),
        min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 20),
        random_state=42
    )
    return evaluate_model_performance(model)

# Evaluate model performance
def evaluate_model_performance(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return mean_squared_error(y_test, y_pred, squared=False)  # RMSE

# Create Optuna studies for each model
studies = {
    'RandomForest': optuna.create_study(direction='minimize'),
    'XGBoost': optuna.create_study(direction='minimize'),
    'DecisionTree': optuna.create_study(direction='minimize')
}

# Optimize hyperparameters for each model
for model_name, study in studies.items():
    if model_name == 'RandomForest':
        study.optimize(tune_random_forest, n_trials=20)
    elif model_name == 'XGBoost':
        study.optimize(tune_xgboost, n_trials=20)
    elif model_name == 'DecisionTree':
        study.optimize(tune_decision_tree, n_trials=20)

    # Print best trial and best hyperparameters
    print(f"Best trial for {model_name}:", study.best_trial)
    print(f"Best hyperparameters for {model_name}:", study.best_params)

# After tuning, create final models with best hyperparameters
final_models = {
    'RandomForest': RandomForestRegressor(**study.best_params, random_state=42),
    'XGBoost': XGBRegressor(**study.best_params, random_state=42),
    'DecisionTree': DecisionTreeRegressor(**study.best_params, random_state=42)
}

# Scale features using MinMaxScaler and fit the final models
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit each final model, make predictions, and evaluate
for model_name, model in final_models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    evaluate_model(model_name, y_test, y_pred)


[I 2024-10-25 02:28:24,977] A new study created in memory with name: no-name-70c28616-8229-41f3-ac59-b46b0b3d0854
[I 2024-10-25 02:28:24,977] A new study created in memory with name: no-name-ef5857d7-ea12-450b-a54e-3abff590eb6a
[I 2024-10-25 02:28:24,977] A new study created in memory with name: no-name-57b7f82d-7ddd-43be-b300-54ee5cb41ba8
/Users/ghanshyamsingh/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:508: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-10-25 02:28:25,458] Trial 0 finished with value: 31466.49989342611 and parameters: {'n_estimators': 87, 'max_depth': 10, 'min_samples_split': 12, 'min_samples_leaf': 4}. Best is trial 0 with value: 31466.49989342611.
/Users/ghanshyamsingh/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:508: FutureWarning: 'squared' is deprecated in version 1.4 a

Best trial for RandomForest: FrozenTrial(number=17, state=1, values=[30363.3988393809], datetime_start=datetime.datetime(2024, 10, 25, 2, 28, 34, 537828), datetime_complete=datetime.datetime(2024, 10, 25, 2, 28, 35, 401569), params={'n_estimators': 128, 'max_depth': 19, 'min_samples_split': 6, 'min_samples_leaf': 3}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=200, log=False, low=50, step=1), 'max_depth': IntDistribution(high=20, log=False, low=2, step=1), 'min_samples_split': IntDistribution(high=20, log=False, low=2, step=1), 'min_samples_leaf': IntDistribution(high=20, log=False, low=1, step=1)}, trial_id=17, value=None)
Best hyperparameters for RandomForest: {'n_estimators': 128, 'max_depth': 19, 'min_samples_split': 6, 'min_samples_leaf': 3}


/Users/ghanshyamsingh/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:508: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-10-25 02:28:38,114] Trial 0 finished with value: 31670.650390625 and parameters: {'n_estimators': 72, 'max_depth': 14, 'learning_rate': 0.21063123859355923}. Best is trial 0 with value: 31670.650390625.
/Users/ghanshyamsingh/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:508: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-10-25 02:28:38,690] Trial 1 finished with value: 34560.16796875 and parameters: {'n_estimators': 167, 'max_depth': 11, 'learning_rate': 0.07282763313945101}. Best is trial 0 with value: 31670.650390625.
/Users/ghanshyam

Best trial for XGBoost: FrozenTrial(number=8, state=1, values=[29459.263671875], datetime_start=datetime.datetime(2024, 10, 25, 2, 28, 40, 180527), datetime_complete=datetime.datetime(2024, 10, 25, 2, 28, 40, 274549), params={'n_estimators': 71, 'max_depth': 5, 'learning_rate': 0.12445779691167969}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=200, log=False, low=50, step=1), 'max_depth': IntDistribution(high=15, log=False, low=3, step=1), 'learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None)}, trial_id=8, value=None)
Best hyperparameters for XGBoost: {'n_estimators': 71, 'max_depth': 5, 'learning_rate': 0.12445779691167969}


/Users/ghanshyamsingh/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:508: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-10-25 02:28:43,340] Trial 16 finished with value: 45453.54934720433 and parameters: {'max_depth': 16, 'min_samples_split': 11, 'min_samples_leaf': 1}. Best is trial 9 with value: 38153.5283914348.
/Users/ghanshyamsingh/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:508: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-10-25 02:28:43,356] Trial 17 finished with value: 38703.372673603735 and parameters: {'max_depth': 10, 'min_samples_split': 20, 'min_samples_leaf': 14}. Best is trial 9 with value: 38153.5283914348.
/Users/ghanshyamsingh/anac

Best trial for DecisionTree: FrozenTrial(number=9, state=1, values=[38153.5283914348], datetime_start=datetime.datetime(2024, 10, 25, 2, 28, 43, 213457), datetime_complete=datetime.datetime(2024, 10, 25, 2, 28, 43, 223864), params={'max_depth': 14, 'min_samples_split': 15, 'min_samples_leaf': 16}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'max_depth': IntDistribution(high=20, log=False, low=1, step=1), 'min_samples_split': IntDistribution(high=20, log=False, low=2, step=1), 'min_samples_leaf': IntDistribution(high=20, log=False, low=1, step=1)}, trial_id=9, value=None)
Best hyperparameters for DecisionTree: {'max_depth': 14, 'min_samples_split': 15, 'min_samples_leaf': 16}


/Users/ghanshyamsingh/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:508: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/ghanshyamsingh/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [02:28:43] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)


RandomForest Performance:
Mean Absolute Error (MAE): 19847.57
Mean Squared Error (MSE): 1282042546.20
Root Mean Squared Error (RMSE): 35805.62
R-squared (R²): 0.82
Explained Variance Score: 0.82


XGBoost Performance:
Mean Absolute Error (MAE): 20075.74
Mean Squared Error (MSE): 1207983616.00
Root Mean Squared Error (RMSE): 34756.06
R-squared (R²): 0.83
Explained Variance Score: 0.83


DecisionTree Performance:
Mean Absolute Error (MAE): 24022.59
Mean Squared Error (MSE): 1455522739.76
Root Mean Squared Error (RMSE): 38151.31
R-squared (R²): 0.80
Explained Variance Score: 0.80




/Users/ghanshyamsingh/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:508: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/ghanshyamsingh/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:508: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
# import pickle
# file_name = 'House_prediction_model.pkl'
# pickle.dump(xgb_r, open(file_name, 'wb'))

In [ ]:
# pipeline = Pipeline(steps =[('random_forest' = rfr)])


# param_grid = 

In [ ]:
# Loading the testing dataset of house price.
df_testing = pd.read_csv('test.csv')

In [ ]:
# df_testing.drop(['Id'], axis = 1)